In [1]:
import wfdb
from wfdb import processing
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib import figure
from scipy.fft import fft, ifft

def load_data(path):
    """Loads database file and returns ECG signal samples and fields list"""
    try:
        signals, fields = wfdb.rdsamp(path)
        ECG = signals[:,0]
        return ECG, fields
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None


def remove_baseline_wander_fft(ekg_data, sampling_rate=4, cutoff_frequency=0.5):
    """Removes the baseline wander for better manipulation with data. Using Fourier Transformation
    to locate lowfrequency components and erasing them."""
    spectrum = fft(ekg_data)
    freq = np.fft.fftfreq(len(ekg_data), 1/sampling_rate)
    spectrum[(freq > -cutoff_frequency) & (freq < cutoff_frequency)] = 0
    ekg_without_baseline = np.real(ifft(spectrum))
    return ekg_without_baseline

def plot(data, samp_rate):
    """Creates a plot. F(t)"""
    time = np.arange(0, len(data) / samp_rate, 1 / samp_rate)

    plt.plot(time, data)
    plt.tight_layout()
    plt.show()


database = 'Database/drive05'
ECGnative, fields = load_data(database)
samp_rate = fields['fs']
print(samp_rate)
#crapped data for rewiev
ecg_data = np.array(ECGnative[:100])

cutoff_frequency = 0.5

ecg_edit_fft = remove_baseline_wander_fft(ecg_data, samp_rate, cutoff_frequency=cutoff_frequency)
plot(ecg_data, samp_rate)
plot(ecg_edit_fft, samp_rate)
print(ecg_edit_fft)



ModuleNotFoundError: No module named 'wfdb'

In [3]:
def detect_peaks(X, N=5, area=3, rthreshold=50, pthreshold=90, do_plot=False):
    WS = (N//2)+1
    f, t, Zxx = scipy.signal,stft(X, nperseg=N, noverlap=0)
    Zxx = np.abs(Zxx).reshape(-1, WS)
    wmax = np.max(Zxx, axis=1)
    thresholdP = np.percentile(X, pthreshold)
    peaks = []
    for base in intervals.astype(int):
        if base >= X.size:
            continue
        if base > WS + area:
            base = base-WS-area
        end = base + N + area*2
        window = X[base:end]
        imax = np.max(window)
        if imax > thresholdP:
            peakT = base + np.argmax(window)
            if peakT in peaks:
                continue
            peaks.append(peakT)